In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\Denish\AppData\Local\Temp\ipykernel_37388\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [3]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv(r"C:\Users\Denish\Desktop\AI\Labs\lab-natural-language-processing\data\kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


In [4]:
data.head()

,text,label
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1
1,Will do.,0
2,Nora--Cheryl has emailed dozens of memos about...,0
3,Dear Sir=2FMadam=2C I know that this proposal ...,1
4,fyi,0


### Let's divide the training and test set into two partitions

In [5]:
# Your code
from sklearn.model_selection import train_test_split


# Original data
X = data[['text']]   # Make X a DataFrame with 'text' column
y = data['label']

# Split the data
data_train, data_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Add labels back to train and val
data_train = data_train.copy()
data_train['label'] = y_train.values

data_val = data_val.copy()
data_val['label'] = y_val.values




## Data Preprocessing

In [6]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [7]:
# Your code
#First we remove inline JavaScript/CSS
import re

def remove_html_javascript(text):
    # Remove <script> or <style> blocks
    text = re.sub(r'<(script|style).*?>.*?</\1>', '', text, flags=re.DOTALL)
    
    # Remove HTML comments
    text = re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)
    
    # Remove remaining HTML tags
    text = re.sub(r'<[^>]+>', '', text)
    
    return text

data_train['text'] = data_train['text'].apply(remove_html_javascript)
data_val['text'] = data_val['text'].apply(remove_html_javascript)




In [8]:
data_train.head()

,text,label
29,"----------- REGARDS, MR NELSON SMITH.KINDLY RE...",1
535,I have not been able to reach oscar this am. W...,0
695,; Huma Abedin B6I'm checking with Pat on the 5...,0
557,I can have it announced here on Monday - can't...,0
836,BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 S...,1


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [9]:
# Your code
import string
import re

def cleaning_text(text):
    # 1. Remove all special characters (keep letters, numbers, and spaces)
    text = ''.join(char if char.isalnum() or char.isspace() else ' ' for char in text)

    # 2. Remove numbers
    text = ''.join(char for char in text if not char.isdigit())

    # Split into words to handle single-character words
    words = text.split()

    # 3. Remove all single-character words
    words = [word for word in words if len(word) > 1]

    # Join back to string
    text = ' '.join(words)

    # 4. Remove single character from the beginning of the entire text if present
    if len(text) > 0 and len(text[0]) == 1 and text[0].isalpha():
        text = text[1:]

    # 5. Substitute multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text).strip()

    # 6. Remove prefixed 'b' if present (like b'example')
    if text.startswith("b'") or text.startswith('b"'):
        text = text[2:-1]

    # 7. Convert to lowercase
    text = text.lower()

    return text

data_train['text'] = data_train['text'].apply(cleaning_text)
data_val['text'] = data_val['text'].apply(cleaning_text)


In [10]:
data_train.head()

,text,label
29,egards mr nelson smith kindly reply me on my p...,1
535,ave not been able to reach oscar this am we ar...,0
695,uma abedin bi checking with pat on the will wo...,0
557,an have it announced here on monday can today,0
836,ank of africaagence san pedro bp san pedro cot...,1


## Now let's work on removing stopwords
Remove the stopwords.

In [11]:
# Your code
#Remove stop words
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    return ' '.join([word for word in text.split() if word not in stop_words])

data_train['text'] = data_train['text'].apply(remove_stopwords)
data_val['text'] = data_val['text'].apply(remove_stopwords)



## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [12]:
# Your 
#Lemmatization
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def lemmatize_text(text):
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

data_train['text'] = data_train['text'].apply(lemmatize_text)
data_val['text'] = data_val['text'].apply(lemmatize_text)


In [13]:
data_train.head()

,text,label
29,egards mr nelson smith kindly reply private em...,1
535,ave able reach oscar supposed send pdb receive,0
695,uma abedin bi checking pat work jack jake rest...,0
557,announced monday today,0
836,ank africaagence san pedro bp san pedro cote i...,1


## Bag Of WordsLet's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)


In [15]:
#Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)
ham_messages = data_train[data_train['label'] == 0]
spam_messages = data_train[data_train['label'] == 1]

print("Number of ham messages:", ham_messages[:10])
print("Number of spam messages:", spam_messages[:10])

Number of ham messages:                                                   text  label
535     ave able reach oscar supposed send pdb receive      0
695  uma abedin bi checking pat work jack jake rest...      0
557                             announced monday today      0
824  ear nancy much want meet trying pin schedule e...      0
65   ust met fr gudziak come back kyiv visit also j...      0
141          word back chinese availability blair fine      0
925                                 hen steinberg call      0
827                                      ugust hillary      0
664  ith permission changing statement spokesman na...      0
907  lso ann sending number people include kc honor...      0
Number of spam messages:                                                   text  label
29   egards mr nelson smith kindly reply private em...      1
836  ank africaagence san pedro bp san pedro cote i...      1
596  ear friend proposal surprising personal contac...      1
165  oneri bank limit

## Extra features

In [16]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€",r"\$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['text'].apply(lambda x: len(x)) 

data_train.head()

,text,label,money_mark,suspicious_words,text_len
29,egards mr nelson smith kindly reply private em...,1,1,0,79
535,ave able reach oscar supposed send pdb receive,0,1,0,46
695,uma abedin bi checking pat work jack jake rest...,0,1,0,78
557,announced monday today,0,1,0,22
836,ank africaagence san pedro bp san pedro cote i...,1,1,1,1050


## How would work the Bag of Words with Count Vectorizer concept?

In [19]:
# Your code
#Bag of Words with count vectroizer
from sklearn.feature_extraction.text import CountVectorizer
# Initialize CountVectorizer        
count_vectorizer = CountVectorizer(max_features=1000)  # Limit to 1000 features for speed
# Fit and transform the training data
X_train_counts = count_vectorizer.fit_transform(data_train['text'])
# Transform the validation data
X_val_counts = count_vectorizer.transform(data_val['text'])
# Convert to DataFrame for better visualization
X_train_counts_df = pd.DataFrame(X_train_counts.toarray(), columns=count_vectorizer.get_feature_names_out())
X_train_counts_df.head()

,ab,abacha,abandoned,abidjan,able,abroad,accept,acceptance,access,accord,...,writing,www,yahoo,year,yet,yi,york,yr,zimbabwe,â½
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,3,1,0,0,0,0,0,0


## TF-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [20]:
# Your code
from sklearn.feature_extraction.text import TfidfVectorizer

# 1. Load the vectorizer and fit on training data (to learn vocabulary and IDF)
vectorizer = TfidfVectorizer()

# 2. Vectorize training data
X_train_tfidf = vectorizer.fit_transform(data_train['text'])

# 3. Vectorize validation data (use transform, NOT fit_transform)
X_val_tfidf = vectorizer.transform(data_val['text'])

# 4. Print shapes
print("Train dataset TF-IDF shape:", X_train_tfidf.shape)
print("Validation dataset TF-IDF shape:", X_val_tfidf.shape)
# 5. Convert to DataFrame for better visualization
X_train_tfidf_df = pd.DataFrame(X_train_tfidf.toarray(), columns=vectorizer.get_feature_names_out())

Train dataset TF-IDF shape: (800, 20275)
Validation dataset TF-IDF shape: (200, 20275)


## And the Train a Classifier?

In [21]:
# Your code
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Assuming your labels are in data_train['label'] and data_val['label']
y_train = data_train['label']
y_val = data_val['label']

# Initialize the classifier
clf = LogisticRegression(max_iter=1000, random_state=42)

# Train the classifier on TF-IDF features
clf.fit(X_train_tfidf, y_train)

# Predict on validation set
y_pred = clf.predict(X_val_tfidf)

# Evaluate
print("Validation Accuracy:", accuracy_score(y_val, y_pred))
print("\nClassification Report:\n", classification_report(y_val, y_pred))


Validation Accuracy: 0.96

Classification Report:
               precision    recall  f1-score   support

           0       0.94      1.00      0.97       125
           1       1.00      0.89      0.94        75

    accuracy                           0.96       200
   macro avg       0.97      0.95      0.96       200
weighted avg       0.96      0.96      0.96       200



### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to **find the most relevant features**.

For example, you can test the following options and check which of them performs better:
- Using "Bag of Words" only
- Using "TF-IDF" only
- Bag of Words + extra flags (money_mark, suspicious_words, text_len)
- TF-IDF + extra flags


You can work with teams of two persons (recommended).

In [22]:
# Your code
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X_train_counts = vectorizer.fit_transform(data_train['text'])
X_val_counts = vectorizer.transform(data_val['text'])


In [24]:
X_train_counts.toarray().shape, X_val_counts.toarray().shape

((800, 20275), (200, 20275))

In [25]:
vectorizer = CountVectorizer(ngram_range=(1,2), min_df=2, max_df=0.9, stop_words='english')
X_train_counts = vectorizer.fit_transform(data_train['text'])
X_val_counts = vectorizer.transform(data_val['text'])


In [26]:
from scipy.sparse import hstack

# Vectorized text
X_train_vec = vectorizer.fit_transform(data_train['text'])
X_val_vec = vectorizer.transform(data_val['text'])

# Numeric features as sparse matrix
import numpy as np
from scipy import sparse

numeric_features_train = data_train[['money_mark', 'suspicious_words', 'text_len']].values
numeric_features_val = data_val[['money_mark', 'suspicious_words', 'text_len']].values

# Convert numeric features to sparse and hstack
X_train_final = hstack([X_train_vec, sparse.csr_matrix(numeric_features_train)])
X_val_final = hstack([X_val_vec, sparse.csr_matrix(numeric_features_val)])


In [28]:
# Initialize the classifier with default params
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()

# Train the model
clf.fit(X_train_final, y_train)

# Predict on validation data
y_pred = clf.predict(X_val_final)

# Evaluate
print("Validation Accuracy:", accuracy_score(y_val, y_pred))
print("\nClassification Report:\n", classification_report(y_val, y_pred))

Validation Accuracy: 0.97

Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.96      0.98       125
           1       0.94      0.99      0.96        75

    accuracy                           0.97       200
   macro avg       0.96      0.97      0.97       200
weighted avg       0.97      0.97      0.97       200

